## Fine-tune Llama for SUD

In [ ]:
!pip3 install -q -U accelerate==0.23.0 peft==0.5.0 bitsandbytes==0.41.1 transformers==4.31 trl==0.7.2
!pip3 install xformers
!pip3 install tensorboardX

## Import libraries

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
import pandas as pd
import os
import csv
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

## Data Loading


In [5]:
# Read data from CSV file into a pandas DataFrame

filename = "/data/ARENAS_Automatic_Extremist_Analysis/ARENAS_Automatic_Extremist_Analysis/Data/SUD_data/all.csv"
df = pd.read_csv(filename)
# Rename columnns
df.rename(columns={'text': 'input', 'class': 'labels'}, inplace=True)
# Select only the "input" and "labels" columns from the DataFrame
df = df[["input", "labels"]]


In [6]:
df

,input,labels
0,as a woman you shouldnt complain about cleanin...,neither
1,boy dats cold tyga dwn bad for cuffin dat hoe ...,offensive
2,dawg you ever fuck a bitch and she sta to cry ...,offensive
3,she look like a tranny,offensive
4,the shit you hear about me might be true or it...,offensive
...,...,...
470763,they belong to you flight diy terrorist countr...,aggressive
470764,really motivating programme congratulations to...,neither
470765,fabricated news,aggressive
470766,whats wrong with you secular idiots,aggressive


In [7]:
# Assuming df is your DataFrame and 'column_name' is the name of the column
unique_values = df['labels'].unique()

# Print the unique values
print(unique_values)

['neither' 'offensive' 'hate' 'abusive' 'profane' 'severe_toxic' 'toxic'
 'identity_hate' 'insult' 'obscene' 'threat' 'aggressive']


In [8]:
# Assuming df is your DataFrame and 'column_name' is the name of the column
df['labels'] = df['labels'].replace({'severe_toxic': 'severe', 'identity_hate': 'identity', "insult": "ins", "offensive": "off", "profane": "prof", "obscene": "obsc", "toxic": "to"})

# Now 'severe_toxic' should be replaced with 'severe toxic' and 'intentity_hate' with 'identity hate'


In [9]:
unique_values = df['labels'].unique()

print(unique_values)

['neither' 'off' 'hate' 'abusive' 'prof' 'severe' 'to' 'identity' 'ins'
 'obsc' 'threat' 'aggressive']


## Data Splitting and Prompt Generation

In [11]:
X_train = list()
X_test = list()
# Split the data into training, testing, and evaluation sets for each sentiment label
for labels in ["off", "neither", "hate", "severe", "to", "ins", "prof", "obsc", "identity", "threat"]:
    train, test  = train_test_split(df[df.labels==labels], 
                                    train_size=344,
                                    test_size=344, 
                                    random_state=42)
    X_train.append(train)
    X_test.append(test)

# Concatenate and shuffle the training and testing sets
X_train = pd.concat(X_train).sample(frac=1, random_state=10)
X_test = pd.concat(X_test)

# Select evaluation samples from the remaining data
eval_idx = [idx for idx in df.index if idx not in list(train.index) + list(test.index)]
X_eval = df[df.index.isin(eval_idx)]
X_eval = (X_eval
          .groupby('labels', group_keys=False)
          .apply(lambda x: x.sample(n=50, random_state=10, replace=True)))

# Reset index of the training set
X_train = X_train.reset_index(drop=True)

# Function to generate prompts for training and evaluation data
def generate_prompt(data_point):
    return f"""
            Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, 
            and return the answer as the corresponding label:
            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat". 
            Make sure to give the whole label as an answer.
            [{data_point["input"]}] = {data_point["labels"]}
            """.strip()

# Function to generate prompts for testing data
def generate_test_prompt(data_point):
    return f"""
            Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, 
            and return the answer as the corresponding label:
            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".
            Make sure to give the whole label as an answer.
            [{data_point["input"]}] = """.strip()

# Convert the prompts into DataFrames for training, evaluation, and testing
X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1), 
                       columns=["input"])
X_eval = pd.DataFrame(X_eval.apply(generate_prompt, axis=1), 
                      columns=["input"])

y_true = X_test.labels
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["input"])

# Create datasets from the generated prompts
train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

In [12]:
labels

'threat'

## Evaluation function

In [13]:
def evaluate(y_true, y_pred):
    # Define labels and their corresponding numeric mappings
    labels = ["off", "neither", "hate", "severe", "to", "ins", "prof", "obsc", "identity", "threat"]
    mapping = {"off": 9, "neither": 8, "hate": 7, "severe" : 6, "to": 5, "ins": 4, "prof": 3, "obsc": 2, "identity": 1, "threat": 0}
    
    # Function to map labels to numeric values
    def map_func(x):
        return mapping.get(x, 1)
    
    # Apply mapping to true and predicted labels
    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')
    
    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels
    
    for label in unique_labels:
        label_indices = [i for i in range(len(y_true)) 
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')
        
    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    print('\nConfusion Matrix:')
    print(conf_matrix)



## Model Configuration and Initialization

In [14]:
#model_name = "../input/llama-2/pytorch/7b-hf/1"
model_name = "meta-llama/Llama-2-7b-hf"
#model_name = "meta-llama/Llama-2-13b-hf"
# Define the data type for model computation
compute_dtype = getattr(torch, "float16")

# Configure BitsAndBytes quantization parameters
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

# Initialize the model with quantization settings
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config, 
)

# Initialize the model with quantization settings
model.config.use_cache = False
model.config.pretraining_tp = 1

# Initialize the tokenizer for the model
tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True,
                                         )
# Set padding token and side for the tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|██████████| 2/2 [02:07<00:00, 63.93s/it]


## Prediction Function


In [17]:
def predict(test, model, tokenizer):
    """
    Make predictions using a text generation model.

    Parameters:
        test (pd.DataFrame): DataFrame containing test data.
        model: Pre-trained text generation model.
        tokenizer: Tokenizer for the model.

    Returns:
        list: Predicted labels for each input.
    """    
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["input"]
        # Create a text generation pipeline
        pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens = 1, 
                        temperature = 0.0,
                       )
        # Generate text based on the prompt
        result = pipe(prompt)
        #print(result)
        answer = result[0]['generated_text'].split("=")[-1]
        #print(answer)
        # Map the generated text to sentiment labels
        if "off" in answer:
            y_pred.append("off")
        elif "hate" in answer:
            y_pred.append("hate")
        elif "severe" in answer:
            y_pred.append("severe")
        elif "to" in answer:
            y_pred.append("to")
        elif "ins" in answer:
            y_pred.append("ins")
        elif "prof" in answer:
            y_pred.append("prof")
        elif "obsc" in answer:
            y_pred.append("obsc")
        elif "identity" in answer:
            y_pred.append("identity")
        elif "threat" in answer:
            y_pred.append("threat")
        else:
            y_pred.append("neither")
    return y_pred

In [ ]:
y_pred = predict(test, model, tokenizer)


In [57]:
evaluate(y_true, y_pred)


Accuracy: 0.100
Accuracy for label 0: 0.000
Accuracy for label 1: 0.000
Accuracy for label 2: 0.000
Accuracy for label 3: 0.000
Accuracy for label 4: 0.000
Accuracy for label 5: 0.000
Accuracy for label 6: 0.000
Accuracy for label 7: 0.000
Accuracy for label 8: 1.000
Accuracy for label 9: 0.000

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00        10
           3       0.00      0.00      0.00        10
           4       0.00      0.00      0.00        10
           5       0.00      0.00      0.00        10
           6       0.00      0.00      0.00        10
           7       0.00      0.00      0.00        10
           8       0.10      1.00      0.18        10
           9       0.00      0.00      0.00        10

    accuracy                           0.10       100
   macro avg       0.01      0

## LoRA Model Configuration

In [21]:
# LoRA Model Configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)
# Training Arguments Configuration
training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=4,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8, # 4
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    evaluation_strategy="epoch"
)

# Trainer Initialization
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="input",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=1024,
)

Map: 100%|██████████| 600/600 [00:00<00:00, 4162.67 examples/s]


## Model training

In [22]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("trained-model")

Epoch,Training Loss,Validation Loss
1,0.814600,0.716811
2,0.672400,0.711314
3,0.633100,0.711217
4,0.559700,0.715227


## Model Predictions

In [23]:
y_pred = predict(test, model, tokenizer)
evaluate(y_true, y_pred)

100%|██████████| 3440/3440 [13:48<00:00,  4.15it/s]

Accuracy: 0.559
Accuracy for label 0: 0.797
Accuracy for label 1: 0.590
Accuracy for label 2: 0.500
Accuracy for label 3: 0.581
Accuracy for label 4: 0.244
Accuracy for label 5: 0.375
Accuracy for label 6: 0.494
Accuracy for label 7: 0.602
Accuracy for label 8: 0.738
Accuracy for label 9: 0.666

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.80      0.77       344
           1       0.53      0.59      0.56       344
           2       0.45      0.50      0.47       344
           3       0.69      0.58      0.63       344
           4       0.44      0.24      0.31       344
           5       0.42      0.38      0.40       344
           6       0.55      0.49      0.52       344
           7       0.45      0.60      0.51       344
           8       0.70      0.74      0.72       344
           9       0.59      0.67      0.63       344

    accuracy                           0.56      3440
   macro avg       0.56      0

## Training log

In [51]:
from sklearn.metrics import classification_report, confusion_matrix

def log_training_results(model_name, bnb_config, peft_config, training_arguments, trainer_state, splitting_info, prompt_generation_info, y_true, y_pred):
    log_file = "training_log.txt"

    # Calculate classification report and confusion matrix
    classification_rep = classification_report(y_true, y_pred)
    confusion_mat = confusion_matrix(y_true, y_pred).tolist()

    # Append the current training information to the log file
    with open(log_file, 'a') as txtfile:
        txtfile.write(f"Model Name: {model_name}\n")
        txtfile.write(f"BitsAndBytes Config: {str(bnb_config)}\n")
        txtfile.write(f"Lora Config: {str(peft_config)}\n")
        txtfile.write(f"Training Arguments: {str(training_arguments)}\n")
        txtfile.write(f"Splitting Info: {splitting_info}\n")
        txtfile.write(f"Prompt Generation Info: {prompt_generation_info}\n")
        txtfile.write(f"Classification Report:\n{classification_rep}\n")
        txtfile.write(f"Confusion Matrix:\n{confusion_mat}\n\n")

    # Return the classification report
    return classification_rep


# Log training parameters, results, splitting info, prompt generation info, and prediction results
splitting_info = "Training samples: {}, Testing samples: {}, Evaluation samples: {}".format(len(X_train), len(X_test), len(X_eval))
prompt_generation_info = "Prompt generation details:  Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, and return the answer as the corresponding label: off or neither or hate or severe or to or ins or prof or obsc or identity or threat. Make sure to give the whole label as an answer."  # Add details about how prompts were generated

# Store the classification report in a text file
classification_rep = log_training_results(model_name, bnb_config, peft_config, training_arguments, trainer.state, splitting_info, prompt_generation_info, y_true, y_pred)

# Now you can access the classification report
print(classification_rep)


              precision    recall  f1-score   support

        hate       0.45      0.60      0.51       344
    identity       0.53      0.59      0.56       344
         ins       0.44      0.24      0.31       344
     neither       0.70      0.74      0.72       344
        obsc       0.45      0.50      0.47       344
         off       0.59      0.67      0.63       344
        prof       0.69      0.58      0.63       344
      severe       0.55      0.49      0.52       344
      threat       0.75      0.80      0.77       344
          to       0.42      0.38      0.40       344

    accuracy                           0.56      3440
   macro avg       0.56      0.56      0.55      3440
weighted avg       0.56      0.56      0.55      3440

